### Initiation

In [1]:
# To manipulate arrays
import numpy as np 
# To import images 

# To handle track DataFrames
import pandas as pd

# To import images
from skimage import io 
from skimage.io import imread

# For TrackPy
import trackpy as tp

# To make plots
import matplotlib as mpl 
import matplotlib.pyplot as plt 
import seaborn as sns; sns.set()  

# To work inline; change to %matplotlib notebook for interactive plotting
%matplotlib inline 

# Napari 
%gui qt5 
from skimage import data
import napari

# To create interactive elements
import ipywidgets as widgets 
from ipywidgets import interact, interactive, fixed, interact_manual, Button, HBox, VBox, Layout, GridspecLayout
from ipywidgets.embed import embed_minimal_html, dependency_state

# Image processing and filters
from skimage.filters import difference_of_gaussians

# Iteration tools such as groupby 
import itertools

# For directories 
import os

# For reloading a library when testing
import importlib

# For deleting/reloading modules in another file for code testing
import sys

# Import trackArrayTools
from trackArrayToolsV2 import *


### Set current working directory, track array filenames, and basic parameters

In [2]:
# Set the current working directory
os.chdir('C:/Users/tim_s/OneDrive - Colostate/Stasevich Lab/Lab Management/Dry Lab/Python/Python Scripts/LabScripts/TestData/')

In [284]:
# Enter in the following:
crop_pad = 5    # specifies the size of crops in track arary. If 5, then crops are (2*5 + 1 = 11) x 11
xy_pixel_size = 130   # voxel  xy dimensions in crops
z_pixel_size = 500   # voxel z dimension
working_directory = 'C:/Users/tim_s/OneDrive - Colostate/Stasevich Lab/Lab Management/Dry Lab/Python/Python Scripts/LabScripts/TestData/'
#video_3D_filename = 'Hela_confocal.tif'
video_3D_filename = 'TA02_90m.tif'
#video_3D_filename = 'TA07_90m.tif'
video_3D_filename_path = 'E:/Tim_tracking videos_all zs/'

In [285]:
#  Track array filenames should be the following:
track_array_filename = 'Tracks_' + video_3D_filename[:-4] + '_crop_pad_' + str(crop_pad) + '.tif' 
track_filename = track_array_filename[:-15] + '.csv'

# For display, so z and xy are shown properly when viewing track array
z_renorm = z_pixel_size/xy_pixel_size  
track_array_filename

'Tracks_TA02_90m_crop_pad_5.tif'

### Creating Track Array (only done once)

In [244]:
#video_3D = imread(video_3D_filename_path+video_3D_filename)

In [266]:
# tracks = pd.read_csv(working_directory + track_filename) 
# tracks.head()

In [267]:
# viewer = napari.Viewer()
# viewer.add_image(video_3D[0::10,:,:,:,0],name='red',blending="additive", scale=[z_renorm,1,1])
# viewer.add_tracks(tracks[['TRACK_ID','POSITION_T','POSITION_Z','POSITION_Y','POSITION_X']].values, name="TRACK_IDs")

In [255]:
# create_track_array_video(working_directory, track_array_filename, video_3D[0::10], tracks, crop_pad, xy_pixel_size, z_pixel_size)

### Read in track array .tif and .csv and do analysis

In [286]:
# Read in the track array video and the tracks files:
track_array_vid = imread(working_directory + track_array_filename)  # Read in track array .tif file
track_array_df = pd.read_csv(working_directory + track_filename) # Read in tracks are read in as "dataframes (df)"
# Make sure track array video channels dimension is listed last
dims = list(track_array_vid.shape)
if len(dims) != 3:     # check if just a single channel video
    n_channels = min(dims)
    n_channels_index = dims.index(n_channels)
    track_array_vid = np.moveaxis(track_array_vid,n_channels_index,-1)

In [287]:
# Create a track array object:
#del sys.modules['trackArrayTools']  # Use this if you are coding new class methods and want to reread trackArrayTools.py file
#from trackArrayToolsV2 import *
ta = TrackArray(track_array_vid, track_array_df, crop_pad)

In [288]:
# Make best_z projection; use -1 offset for blue and green channels... 
my_best_z_masks = ta.z_masks(1,0,z_offset = [0,-1,-1])
best_z_projection = ta.mask_projection(ta.arr, ta.to_crop_array(my_best_z_masks))
cigar_masks = ta.cigar_masks(3,3,2,0)  # make cigar masks centered on tracked x and y and best-z
capsule_masks = ta.capsule_masks(3,3,2,0)  # make capsule masks centered on tracked x and y and best-z 
[ta.to_crop_array(my_best_z_masks).shape,ta.arr.shape,best_z_projection.shape]

[(10, 33, 616, 3), (10, 33, 616, 3), (33, 616, 3)]

In [275]:
# Check offset masks
offset_mask = ta.to_crop_array(my_best_z_masks)[:,:,:,1] # can change to 0 to see non offset mask
ta.napari_viewer(ta.arr,[z_renorm,1,1],int_range = ta.int_range(0.1,8),layer = offset_mask)

In [289]:
# Do local background subtraction, as calculated in capsules
bg_crops = ta.local_background_subtract(capsule_masks)
best_z_projection_bg = ta.mask_projection(ta.to_crop_array(bg_crops),ta.to_crop_array(my_best_z_masks))
kymograph = np.max(bg_crops*my_best_z_masks,axis=2)
[best_z_projection_bg.shape, kymograph.shape]

[(33, 616, 3), (3, 56, 11, 11, 3)]

In [290]:
# Function to do moving average by n frames
def moving_avg(crops,n):
    crops_ma = crops*0
    for i in np.arange(crops.shape[1]-n):
        crops_ma[:,i] = np.mean(crops[:,i:i+n],axis=1)
    return crops_ma

In [291]:
# Do a 3-frame moving average
mov_avg = moving_avg(bg_crops,3)
mov_avg_z_projection = ta.mask_projection(ta.to_crop_array(mov_avg),ta.to_crop_array(my_best_z_masks))
mov_avg_kymograph = np.max(mov_avg,axis=2)
[mov_avg.shape,mov_avg_z_projection.shape,mov_avg_kymograph.shape]

[(3, 56, 10, 11, 11, 3), (33, 616, 3), (3, 56, 11, 11, 3)]

In [263]:
# Check moving average
ta.napari_viewer(mov_avg_z_projection,[1,1],int_range = ta.int_range(3,3)) ## nice visualization

In [292]:
# Now do 2D disk-donut on mov_avg_z_projection to remove nearly all background
my_donut = ta.to_color_mask(my_capsule(crop_pad,crop_pad,0,3,3,1,ta.crop_dim(),ta.crop_dim(),1))[0]
my_disk = ta.to_color_mask(my_cigar(crop_pad,crop_pad,0,3,3,1,ta.crop_dim(),ta.crop_dim(),1))[0]
disk_donut = mov_avg_z_projection*0
crop_dim = ta.crop_dim()
for n in np.arange(ta.n_tracks()):
    for t in np.arange(ta.n_frames()):
        cur_crop = mov_avg_z_projection_crops[n,t]
        bg = np.mean(np.ma.masked_equal((my_donut*cur_crop),0), axis = (0,1)).data
        for ch in np.arange(ta.n_channels()):
            disk_donut[n*crop_dim:n*crop_dim+crop_dim,
                       t*crop_dim:t*crop_dim+crop_dim,ch] = cur_crop[:,:,ch] - bg[ch]        

In [265]:
# Check disk-donut final correction:
max = np.max(disk_donut,axis=(0,1))
ta.napari_viewer(disk_donut,[1,1],int_range=[[0,max[0]],[0,max[1]],[0,max[2]]]) ## best visualization

In [293]:
working_directory + track_array_filename[0:-4] + '_bestz_bg-sub.tif'

'C:/Users/tim_s/OneDrive - Colostate/Stasevich Lab/Lab Management/Dry Lab/Python/Python Scripts/LabScripts/TestData/Tracks_TA02_90m_crop_pad_5_bestz_bg-sub.tif'

In [294]:
# Output background-subtracted and max-z projected track array 
output_filename_path = working_directory + track_array_filename[0:-4] + '_bestz_bg-sub.tif'
io.imsave(output_filename_path,
        disk_donut, 
        resolution=(1/xy_pixel_size,1/xy_pixel_size))

### Read in all final track arrays tifs and combine into a single tif

In [315]:
# Specify files you want to read in
vid1 = 'TA02_90m.tif'
vid2 = 'TA07_90m.tif'
track_array_filename1 = 'Tracks_' + vid1[:-4] + '_crop_pad_' + str(crop_pad) + '_bestz_bg-sub.tif'
track_array_filename2 = 'Tracks_' + vid2[:-4] + '_crop_pad_' + str(crop_pad) + '_bestz_bg-sub.tif'

In [349]:
# Read in the track array video and the tracks files:
track_array_final1 = imread(working_directory + track_array_filename1)  # Read in track array .tif file
track_array_final2 = imread(working_directory + track_array_filename2)  # Read in track array .tif file

In [362]:
dims = np.array([track_array_final1.shape, track_array_final2.shape])
dims

array([[ 33, 616,   3],
       [ 11, 594,   3]])

In [380]:
[sum(dims[:,0]),np.max(dims[:,1])]

[44, 616]

In [389]:
all_tracks = np.zeros((sum(dims[:,0]),np.max(dims[:,1]),dims[0,2]))    
all_tracks[0:dims[0,0],:,:]=track_array_final1
all_tracks[dims[0,0]:dims[0,0]+dims[1,0],0:dims[1,1],:]=track_array_final2

In [393]:
max = np.max(all_tracks,axis=(0,1))
ta.napari_viewer(all_tracks,[1,1],int_range=[[0,max[0]],[0,max[1]],[0,max[2]]]) ## best visualization)

In [392]:
# Output background-subtracted and max-z projected all track arrays 
output_filename_path = working_directory + 'TA0n_90m_bestz_bg-sub_all.tif'
io.imsave(output_filename_path,
        all_tracks, 
        resolution=(1/xy_pixel_size,1/xy_pixel_size))